# FPL data engeeniering

In [14]:
# RELOAD MODULES (useful during development)
import importlib
from utils import data_loader
importlib.reload(data_loader)

<module 'utils.data_loader' from 'D:\\Studies\\Informatyka\\Semestr_3\\FPL_AI_manager\\utils\\data_loader.py'>

In [58]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from utils.data_loader import load_data

Import data and prepare it for MLP model

## 2020-21 season data engineering for baseline model
Predict next game total_points based only on single previous game

In [190]:
data_2021 = load_data('2020-21')

In [191]:
# add 'total_points_next_gameweek' column where total_points_next_gameweek = total_points from next 'GW' for each 'element'
data_2021['total_points_next_gameweek'] = data_2021.groupby('element')['total_points'].shift(-1)

In [192]:
# preview data with 'Adam Lallana' name sorted by 'GW'
data_2021[data_2021['name'] == 'Adam Lallana'].sort_values('GW').head(10)

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,total_points_next_gameweek
8,Adam Lallana,MID,Brighton,1.2,0,0,6,0,27.2,54,...,2.0,1,0,0,0,65,True,0,1,1.0
434,Adam Lallana,MID,Brighton,0.4,0,0,3,0,1.2,54,...,0.0,1,-25783,2185,27968,65,False,0,2,2.0
973,Adam Lallana,MID,Brighton,0.3,0,0,7,0,2.8,54,...,0.0,2,-13848,1340,15188,64,True,0,3,1.0
1527,Adam Lallana,MID,Brighton,0.7,0,0,5,0,13.0,54,...,8.0,1,-5216,1748,6964,64,False,0,4,2.0
2089,Adam Lallana,MID,Brighton,1.5,0,0,4,0,4.9,54,...,1.0,2,-6062,1191,7253,63,False,0,5,3.0
2806,Adam Lallana,MID,Brighton,2.5,0,0,14,1,22.1,54,...,22.0,3,-2589,1019,3608,63,True,0,6,2.0
3396,Adam Lallana,MID,Brighton,1.8,0,0,4,0,16.1,54,...,18.0,2,-2018,813,2831,63,False,0,7,3.0
3995,Adam Lallana,MID,Brighton,2.5,0,0,20,1,43.6,54,...,44.0,3,-1005,1667,2672,62,True,0,8,4.0
4595,Adam Lallana,MID,Brighton,3.0,1,0,18,0,11.1,54,...,6.0,4,-1276,1300,2576,62,False,0,9,1.0
5196,Adam Lallana,MID,Brighton,1.3,0,0,3,0,0.0,54,...,0.0,1,-2472,269,2741,62,True,0,10,0.0


**NOTE:** Model won't have any information about for which player the prediction is made.

In [193]:
# create data_2021_processed dataframe without 'name', 'team', 'fixture', 'kickoff_time', 'opponent_team', 'round', 'team_h_score', 'team_a_score'
data_2021_processed = data_2021.drop(['name', 'team', 'fixture', 'kickoff_time', 'opponent_team', 'round', 'team_h_score', 'team_a_score'], axis=1)

In [194]:
# extract 'GW', 'element' and 'total_points_next_gameweek' from data_2021_processed
data_2021_extract_target = data_2021_processed[['GW', 'element', 'total_points_next_gameweek']]

In [195]:
# drop 'GW' and 'element' from data_2021_processed
data_2021_processed = data_2021_processed.drop(['GW', 'element'], axis=1)

In [196]:
# one-hot encode 'position' column
data_2021_processed = pd.get_dummies(data_2021_processed, columns=['position'])

In [197]:
# change 'was_home' column to binary
data_2021_processed['was_home'] = data_2021_processed['was_home'].map({True: 1, False: 0})

In [198]:
# drop rows with NaN values
data_2021_processed = data_2021_processed.dropna()

In [199]:
data_2021_processed.head(10)

,xP,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,...,transfers_in,transfers_out,value,was_home,yellow_cards,total_points_next_gameweek,position_DEF,position_FWD,position_GK,position_MID
0,0.5,0,0,-3,0,0.3,2,0,3.4,1.2,...,0,0,55,1,0,8.0,0,1,0,0
1,2.1,0,0,11,0,11.2,2,0,2.2,10.4,...,0,0,50,1,0,1.0,1,0,0,0
2,0.0,0,0,0,0,0.0,0,0,0.0,0.0,...,0,0,50,1,0,0.0,0,0,0,1
3,2.5,0,0,12,0,0.0,2,0,1.8,18.2,...,0,0,50,1,0,2.0,0,0,1,0
4,1.3,0,0,20,1,44.6,0,0,6.9,20.4,...,0,0,55,0,0,1.0,0,0,0,1
5,0.4,0,0,-2,0,1.8,3,0,0.2,0.0,...,0,0,50,1,0,1.0,0,0,0,1
6,2.2,0,0,6,1,1.8,0,0,3.2,12.8,...,0,0,65,0,0,2.0,0,0,0,1
7,0.0,0,0,0,0,0.0,0,0,0.0,0.0,...,0,0,50,0,0,0.0,0,0,0,1
8,1.2,0,0,6,0,27.2,1,0,3.6,6.8,...,0,0,65,1,0,1.0,0,0,0,1
9,0.6,0,0,14,0,11.8,3,0,2.8,14.8,...,0,0,45,1,0,6.0,1,0,0,0


In [200]:
X_2021 = data_2021_processed.drop(['total_points_next_gameweek'], axis=1)
y_2021 = data_2021_processed['total_points_next_gameweek']

In [201]:
# scale X data
X_scaler = MinMaxScaler(feature_range=(0, 1))
X_2021_scaled = pd.DataFrame(X_scaler.fit_transform(X_2021), columns = X_2021.columns)

In [202]:
# split data into train and test sets
X_train_2021, X_test_2021, y_train_2021, y_test_2021 = train_test_split(X_2021_scaled, y_2021, test_size=0.2, random_state=42)

### Train on baseline SVR model

In [203]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [204]:
model_svr = SVR()
model_svr.fit(X_train_2021, y_train_2021)

SVR()

In [205]:
# predict on test set
y_pred_2021 = model_svr.predict(X_test_2021)

In [206]:
mean_squared_error(y_pred_2021, y_test_2021)

5.1224767586306035

In [207]:
results = pd.DataFrame(y_test_2021)

In [208]:
results['prediction'] = y_pred_2021

In [211]:
results

,total_points_next_gameweek,prediction
11387,2.0,1.715704
2055,1.0,1.886470
18887,0.0,0.082232
3353,0.0,0.093830
11466,1.0,1.091189
...,...,...
22524,0.0,0.100445
2998,1.0,0.844867
12020,0.0,0.084547
7306,0.0,0.092959
